In [459]:
from nltk.tokenize import sent_tokenize, TweetTokenizer
from string import punctuation
from os import scandir
from collections import defaultdict
from random import random, choice, seed as seed_random, shuffle

tokenizer = TweetTokenizer()

def get_tokens_from_file(path):
    content = open(path).read()
    sentences = sent_tokenize(content)
    tokens = []
    for _sent in sentences:
        sent_tokens = tokenizer.tokenize(_sent)
        sent_tokens = [_tok.lower() for _tok in sent_tokens if _tok not in punctuation]
        tokens += sent_tokens
    return tokens


def calc_bigram_freq(sorted_tokens):
    result = defaultdict(lambda: defaultdict(lambda: 0))
    for f, s in zip(sorted_tokens, sorted_tokens[1:]):
        result[f][s] += 1
    return {k: dict(v) for k, v in result.items()}


def calc_bigram_prob(sorted_tokens):
    bigram_freq = calc_bigram_freq(sorted_tokens)
    return {f: {s: freq / sum(map(lambda x: x[1], goto.items())) for s, freq in goto.items()} 
               for f, goto in bigram_freq.items()}


def prob_walk(seed_word, gen_size, bigram_prob, seed=None):
    def prob_choice(items, key=lambda x: x, seed=None):
        seed_random(seed)
        weights = map(key, items)
        total = sum(weights)
        s = 0
        for i, w in enumerate(weights):
            if s >= random():
                return items[i]
            s += w
        return choice(list(items))

    result = [seed_word]
    for _ in range(gen_size):
        new_word = prob_choice(bigram_prob[result[-1]].items(), key=lambda x: x[1], seed=seed)
        result.append(new_word[0])
    return result

def find_rhyming(word, tokens, seed):
    seed_random(seed)
    shuffle(tokens)
    for n in range(4, 0, -1):
        for token in tokens:
            if token != word and token.endswith(word[-n:]):
                return token
    return choice(tokens)


def generate_rhyming_biverse(seed_word, tokens, forward_bigrams, backwards_bigrams, first_size, second_size, rnd_seed):
    first_line = prob_walk(seed_word, first_size, forward_bigrams, rnd_seed)
    rhyming_word = find_rhyming(first_line[-1], tokens, rnd_seed)
    second_line = prob_walk(rhyming_word, second_size, backwards_bigrams, rnd_seed)
    
    return ' '.join(first_line) + '\n' + ' '.join(second_line[::-1])


class RhymingHMM:
    def fit(self, sorted_tokens):
        self.sorted_tokens = sorted_tokens
        self.bigram_prob = calc_bigram_prob(sorted_tokens)
        self.bigram_prob_rev = calc_bigram_prob(sorted_tokens[::-1])
        return self

    def generate_multiple_rhyming_biverses(self, size, seed_word, line_sizes, rnd_seed):
        first_line_size, second_line_size = line_sizes
        for _ in range(size):
            yield generate_rhyming_biverse(
                seed_word, self.sorted_tokens, self.bigram_prob, self.bigram_prob_rev,
                first_line_size, second_line_size, rnd_seed
            )

            seed_random(rnd_seed)
            seed_word = choice(self.sorted_tokens)

    def rhyme(self, size, seed_word, line_sizes, rnd_seed):
        result = self.generate_multiple_rhyming_biverses(size, seed_word, line_sizes, rnd_seed)
        return '\n'.join(result)

In [418]:
sorted_tokens = get_tokens_from_file('data/smirnenski.txt')
sorted_tokens[:10]

['нощта', 'е', 'черна', 'и', 'зловеща', 'нощта', 'е', 'ледна', 'като', 'смърт']

In [419]:
print(len(sorted_tokens), len(set(sorted_tokens)))

3644 1673


In [420]:
bigram_prob = calc_bigram_prob(sorted_tokens)
bigram_prob_rev = calc_bigram_prob(sorted_tokens[::-1])

In [421]:
list(bigram_freq_dict.items())[50:60]

[('звезда', {'и': 1, 'кървавата': 1, 'на': 1, 'се': 1}),
 ('води', {'подаде': 1}),
 ('издигат', {'на': 1}),
 ('бронята', {'нощите': 1}),
 ('назад', {'делничните': 1, 'скрил': 1}),
 ('учуди', {'съдбоносен': 1}),
 ('синове', {'вбраздени': 1, 'те': 1}),
 ('развява', {'до': 1}),
 ('край', {'о': 1, '—': 1}),
 ('верига', {'пот': 1})]

In [477]:
sorted(bigram_freq_dict.items(), key=lambda x: max(map(lambda y: y[1], x[1].items())))[-10]

('му',
 {'буря': 1,
  'вечерта': 1,
  'гърми': 1,
  'и': 3,
  'призрачни': 1,
  'първи': 1,
  'слънцето': 1,
  'шумни': 1})

In [423]:
list(bigram_prob.items())[10:20]

[('силует', {'на': 1.0}),
 ('отговарят', {'чрез': 1.0}),
 ('сребросинкави', {'петна': 1.0}),
 ('улицата', {'със': 0.5, 'шумна': 0.5}),
 ('строга', {'на': 1.0}),
 ('хладната', {'си': 1.0}),
 ('озарен', {'от': 0.5, 'с': 0.5}),
 ('стозвучно', {'загърмя': 1.0}),
 ('мощ',
  {'горите': 0.3333333333333333,
   'заканата': 0.3333333333333333,
   'слепци': 0.3333333333333333}),
 ('друго', {'рухват': 1.0})]

In [424]:
generated_tokens = prob_walk('улицата', 20, bigram_prob, 5)

In [425]:
print(" ".join(generated_tokens))

улицата със поглед замечтан навън цареше смъртна пустота и стене глухо прибледнелите уста а ний нов път хусарите сред дима на


In [426]:
generated_tokens = prob_walk('деца', 15, bigram_prob, 0)

In [427]:
print(' '.join(generated_tokens))

деца на черна нерадост възмъжах и колко скръб в сърцата пролетарски свят спартаков път над тях


In [429]:
find_rhyming('улицата', sorted_tokens, 0)

'сърцата'

---

In [431]:
prob_walk('деца', 4, bigram_prob, 0)

['деца', 'на', 'черна', 'нерадост', 'възмъжах']

In [321]:
find_rhyming('възмъжах', sorted_tokens)

'размах'

In [413]:
print(generate_rhyming_biverse('и', sorted_tokens, bigram_prob, bigram_prob_rev, 5, 4, 1))

и стиснала детенцето си да възсияй
света и неговия лъч сияй


In [435]:
model = RhymingHMM().fit(sorted_tokens)

In [479]:
print(model.rhyme(4, 'цветарка', (5, 4), 0))

цветарка копняла на душа над очите
ще елате пръст метежна кървавите
ужас зловеща над очите него морни
ръцете дебне непознати е беломраморни
блатата стихва безшумно нужно днес тръгват
вълни навън ужаса в трепват
стозвучний спуснат години ти а нощта
навън ужаса в берлин свойта
